<a href="https://colab.research.google.com/github/Nidharshana0325/NIDHI-IIT-GUWAHATI-DATAHACK/blob/main/NIDHARSHANA__R_IIT_GUWAHATI_HACKATHON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Load training set features and labels from CSV
train_features = pd.read_csv('training_set_features.csv')
train_labels = pd.read_csv('training_set_labels.csv')

# Load test set features from CSV
test_features = pd.read_csv('test_set_features.csv')

# Merge training features and labels
train_df = pd.merge(train_features, train_labels, on='respondent_id')

# Identify categorical columns
categorical_cols = ['age_group', 'education', 'race', 'sex', 'income_poverty',
                    'marital_status', 'rent_or_own', 'employment_status',
                    'hhs_geo_region', 'census_msa', 'employment_industry',
                    'employment_occupation']

# Identify numerical columns
numerical_cols = [col for col in train_df.columns if col not in categorical_cols + ['respondent_id', 'xyz_vaccine', 'seasonal_vaccine']]

# Define transformers for preprocessing
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Apply preprocessing to the training data
X = train_df.drop(['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'], axis=1)
y_xyz = train_df['xyz_vaccine']
y_seasonal = train_df['seasonal_vaccine']

X_encoded = preprocessor.fit_transform(X)

# Split data into training and validation sets
X_train, X_val, y_train_xyz, y_val_xyz = train_test_split(X_encoded, y_xyz, test_size=0.2, random_state=42)
X_train, X_val, y_train_seasonal, y_val_seasonal = train_test_split(X_encoded, y_seasonal, test_size=0.2, random_state=42)

# Initialize logistic regression models
model_xyz = LogisticRegression(max_iter=1000)
model_seasonal = LogisticRegression(max_iter=1000)

# Fit models
model_xyz.fit(X_train, y_train_xyz)
model_seasonal.fit(X_train, y_train_seasonal)

# Predict probabilities for validation set
pred_proba_xyz_val = model_xyz.predict_proba(X_val)[:, 1]
pred_proba_seasonal_val = model_seasonal.predict_proba(X_val)[:, 1]

# Calculate ROC AUC scores for validation set
auc_xyz = roc_auc_score(y_val_xyz, pred_proba_xyz_val)
auc_seasonal = roc_auc_score(y_val_seasonal, pred_proba_seasonal_val)

print(f'ROC AUC for xyz_vaccine: {auc_xyz:.4f}')
print(f'ROC AUC for seasonal_vaccine: {auc_seasonal:.4f}')

# Apply preprocessing to the test data
X_test = test_features.drop(['respondent_id'], axis=1)
X_test_encoded = preprocessor.transform(X_test)

# Predict probabilities for test set
pred_proba_xyz_test = model_xyz.predict_proba(X_test_encoded)[:, 1]
pred_proba_seasonal_test = model_seasonal.predict_proba(X_test_encoded)[:, 1]

# Prepare submission file
submission_df = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],
    'xyz_vaccine': pred_proba_xyz_test,
    'seasonal_vaccine': pred_proba_seasonal_test
})

# Save submission file
submission_df.to_csv('submission_format.csv', index=False)

print("Submission saved successfully.")


ROC AUC for xyz_vaccine: 0.8344
ROC AUC for seasonal_vaccine: 0.8564
Submission saved successfully.
